In [1]:
import datetime as dt
import pandas as pd

time = pd.Series (['2015-03-08 01:00:00',
'2015-03-08 01:30:00',
'2015-03-08 02:00:00',
'2015-03-08 02:30:00',
'2015-03-08 03:00:00',
'2015-03-08 02:00:00',
'2015-03-08 02:30:00',
'2015-03-08 03:00:00',
'2015-03-08 03:30:00',
'2015-11-01 00:30:00',
'2015-11-01 01:00:00',
'2015-11-01 01:30:00',
'2015-11-01 02:00:00',
'2015-11-01 02:30:00',
'2015-11-01 01:00:00',
'2015-11-01 01:30:00',
'2015-11-01 02:00:00',
'2015-11-01 02:30:00', 
'2015-11-01 03:00:00'])

In [5]:
# Offsets in integers get translated as seconds when doing an offset
offset = pd.Series([-7, -7, -7, -7, -7, -6, -6, -6, -6, -6, -6, -6, -6, -6, -7, -7, -7, -7, -7])

In [21]:
# Bugged version
(pd.to_datetime(time) # converts the string series into workable time
 .groupby(offset) # puts the offsets -6 and -7 as bins in which the times fall into
 .transform(lambda s: # takes the lambda funciton and returns the same length series
            s.dt.tz_localize(s.name) # s.name is the group key (timezone offset in this case) # tz_localize attaches the offset as seconds
            .dt.tz_convert('America/Denver'))) # lz.convert then takes the offsets and converts them into the specified timezone


# why it's bugged: because the raw numbers get converted into seconds, not hours.

0    2015-03-07 18:00:07-07:00
1    2015-03-07 18:30:07-07:00
2    2015-03-07 19:00:07-07:00
3    2015-03-07 19:30:07-07:00
4    2015-03-07 20:00:07-07:00
5    2015-03-07 19:00:06-07:00
6    2015-03-07 19:30:06-07:00
7    2015-03-07 20:00:06-07:00
8    2015-03-07 20:30:06-07:00
9    2015-10-31 18:30:06-06:00
10   2015-10-31 19:00:06-06:00
11   2015-10-31 19:30:06-06:00
12   2015-10-31 20:00:06-06:00
13   2015-10-31 20:30:06-06:00
14   2015-10-31 19:00:07-06:00
15   2015-10-31 19:30:07-06:00
16   2015-10-31 20:00:07-06:00
17   2015-10-31 20:30:07-06:00
18   2015-10-31 21:00:07-06:00
dtype: datetime64[ns, America/Denver]

In [19]:
for key, group in pd.to_datetime(time).groupby(offset):
    print(f'Offset: {key}')
    print(group)
    print('------')

Offset: -7
0    2015-03-08 01:00:00
1    2015-03-08 01:30:00
2    2015-03-08 02:00:00
3    2015-03-08 02:30:00
4    2015-03-08 03:00:00
14   2015-11-01 01:00:00
15   2015-11-01 01:30:00
16   2015-11-01 02:00:00
17   2015-11-01 02:30:00
18   2015-11-01 03:00:00
dtype: datetime64[ns]
------
Offset: -6
5    2015-03-08 02:00:00
6    2015-03-08 02:30:00
7    2015-03-08 03:00:00
8    2015-03-08 03:30:00
9    2015-11-01 00:30:00
10   2015-11-01 01:00:00
11   2015-11-01 01:30:00
12   2015-11-01 02:00:00
13   2015-11-01 02:30:00
dtype: datetime64[ns]
------


In [37]:
# Fixed version:
offset = pd.Series([-7, -7, -7, -7, -7, -6, -6, -6, -6, -6, -6, -6, -6, -6, -7, -7, -7, -7, -7])
offset = offset.replace({-7: "Etc/GMT+7", -6: "Etc/GMT+6"}) # explicit statement of offset as hours

local = (pd.to_datetime(time)
         .groupby(offset)
         .transform(lambda s: 
                    s.dt.tz_localize(s.name)
                    .dt.tz_convert('America/Denver')))

local

0    2015-03-08 01:00:00-07:00
1    2015-03-08 01:30:00-07:00
2    2015-03-08 03:00:00-06:00
3    2015-03-08 03:30:00-06:00
4    2015-03-08 04:00:00-06:00
5    2015-03-08 01:00:00-07:00
6    2015-03-08 01:30:00-07:00
7    2015-03-08 03:00:00-06:00
8    2015-03-08 03:30:00-06:00
9    2015-11-01 00:30:00-06:00
10   2015-11-01 01:00:00-06:00
11   2015-11-01 01:30:00-06:00
12   2015-11-01 01:00:00-07:00
13   2015-11-01 01:30:00-07:00
14   2015-11-01 01:00:00-07:00
15   2015-11-01 01:30:00-07:00
16   2015-11-01 02:00:00-07:00
17   2015-11-01 02:30:00-07:00
18   2015-11-01 03:00:00-07:00
dtype: datetime64[ns, America/Denver]

In [38]:
local.dt.tz_convert('UTC')

0    2015-03-08 08:00:00+00:00
1    2015-03-08 08:30:00+00:00
2    2015-03-08 09:00:00+00:00
3    2015-03-08 09:30:00+00:00
4    2015-03-08 10:00:00+00:00
5    2015-03-08 08:00:00+00:00
6    2015-03-08 08:30:00+00:00
7    2015-03-08 09:00:00+00:00
8    2015-03-08 09:30:00+00:00
9    2015-11-01 06:30:00+00:00
10   2015-11-01 07:00:00+00:00
11   2015-11-01 07:30:00+00:00
12   2015-11-01 08:00:00+00:00
13   2015-11-01 08:30:00+00:00
14   2015-11-01 08:00:00+00:00
15   2015-11-01 08:30:00+00:00
16   2015-11-01 09:00:00+00:00
17   2015-11-01 09:30:00+00:00
18   2015-11-01 10:00:00+00:00
dtype: datetime64[ns, UTC]

In [42]:
# convert from UTC to nano seconds using UNIX epoch code

nano_secs = local.astype('int64[pyarrow]')
nano_secs

0     1425801600000000000
1     1425803400000000000
2     1425805200000000000
3     1425807000000000000
4     1425808800000000000
5     1425801600000000000
6     1425803400000000000
7     1425805200000000000
8     1425807000000000000
9     1446359400000000000
10    1446361200000000000
11    1446363000000000000
12    1446364800000000000
13    1446366600000000000
14    1446364800000000000
15    1446366600000000000
16    1446368400000000000
17    1446370200000000000
18    1446372000000000000
dtype: int64[pyarrow]

In [43]:
# converting back to UTC
(pd.to_datetime(nano_secs, unit='ns')
 .dt.tz_localize('UTC'))


0    2015-03-08 08:00:00+00:00
1    2015-03-08 08:30:00+00:00
2    2015-03-08 09:00:00+00:00
3    2015-03-08 09:30:00+00:00
4    2015-03-08 10:00:00+00:00
5    2015-03-08 08:00:00+00:00
6    2015-03-08 08:30:00+00:00
7    2015-03-08 09:00:00+00:00
8    2015-03-08 09:30:00+00:00
9    2015-11-01 06:30:00+00:00
10   2015-11-01 07:00:00+00:00
11   2015-11-01 07:30:00+00:00
12   2015-11-01 08:00:00+00:00
13   2015-11-01 08:30:00+00:00
14   2015-11-01 08:00:00+00:00
15   2015-11-01 08:30:00+00:00
16   2015-11-01 09:00:00+00:00
17   2015-11-01 09:30:00+00:00
18   2015-11-01 10:00:00+00:00
dtype: datetime64[ns, UTC]

In [45]:
# nano seconds (ns) is the wrong unit
(nano_secs
 .truediv(1_000_000)
 .pipe(pd.to_datetime, unit='ns')
 .dt.tz_localize('UTC'))

0    1970-01-01 00:23:45.801600+00:00
1    1970-01-01 00:23:45.803400+00:00
2    1970-01-01 00:23:45.805200+00:00
3    1970-01-01 00:23:45.807000+00:00
4    1970-01-01 00:23:45.808800+00:00
5    1970-01-01 00:23:45.801600+00:00
6    1970-01-01 00:23:45.803400+00:00
7    1970-01-01 00:23:45.805200+00:00
8    1970-01-01 00:23:45.807000+00:00
9    1970-01-01 00:24:06.359400+00:00
10   1970-01-01 00:24:06.361200+00:00
11   1970-01-01 00:24:06.363000+00:00
12   1970-01-01 00:24:06.364800+00:00
13   1970-01-01 00:24:06.366600+00:00
14   1970-01-01 00:24:06.364800+00:00
15   1970-01-01 00:24:06.366600+00:00
16   1970-01-01 00:24:06.368400+00:00
17   1970-01-01 00:24:06.370200+00:00
18   1970-01-01 00:24:06.372000+00:00
dtype: datetime64[ns, UTC]

In [47]:
# microseconds (ms) is the right unit

(nano_secs
 .truediv(1_000_000)
 .pipe(pd.to_datetime, unit='ms')
 .dt.tz_localize('UTC'))

0    2015-03-08 08:00:00+00:00
1    2015-03-08 08:30:00+00:00
2    2015-03-08 09:00:00+00:00
3    2015-03-08 09:30:00+00:00
4    2015-03-08 10:00:00+00:00
5    2015-03-08 08:00:00+00:00
6    2015-03-08 08:30:00+00:00
7    2015-03-08 09:00:00+00:00
8    2015-03-08 09:30:00+00:00
9    2015-11-01 06:30:00+00:00
10   2015-11-01 07:00:00+00:00
11   2015-11-01 07:30:00+00:00
12   2015-11-01 08:00:00+00:00
13   2015-11-01 08:30:00+00:00
14   2015-11-01 08:00:00+00:00
15   2015-11-01 08:30:00+00:00
16   2015-11-01 09:00:00+00:00
17   2015-11-01 09:30:00+00:00
18   2015-11-01 10:00:00+00:00
dtype: datetime64[ns, UTC]

In [52]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/alta-noaa-1980-2019.csv'
alta_df = pd.read_csv(url)
alta_df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,DAPR,DASF,MDPR,MDSF,...,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11
0,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-01,NaN,NaN,NaN,NaN,...,29.0,38.0,25.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-02,NaN,NaN,NaN,NaN,...,34.0,27.0,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-03,NaN,NaN,NaN,NaN,...,30.0,27.0,12.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-04,NaN,NaN,NaN,NaN,...,30.0,31.0,18.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,1980-01-05,NaN,NaN,NaN,NaN,...,30.0,34.0,26.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-03,NaN,NaN,NaN,NaN,...,0.0,74.0,57.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN
14156,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-04,NaN,NaN,NaN,NaN,...,0.0,77.0,52.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN
14157,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-05,NaN,NaN,NaN,NaN,...,0.0,76.0,54.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN
14158,USC00420072,"ALTA, UT US",40.5905,-111.6369,2660.9,2019-09-06,NaN,NaN,NaN,NaN,...,0.0,66.0,52.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
dates = (pd.to_datetime(alta_df.DATE)
         .astype('timestamp[ns][pyarrow]'))
dates

0        1980-01-01 00:00:00
1        1980-01-02 00:00:00
2        1980-01-03 00:00:00
3        1980-01-04 00:00:00
4        1980-01-05 00:00:00
                ...         
14155    2019-09-03 00:00:00
14156    2019-09-04 00:00:00
14157    2019-09-05 00:00:00
14158    2019-09-06 00:00:00
14159    2019-09-07 00:00:00
Name: DATE, Length: 14160, dtype: timestamp[ns][pyarrow]

In [54]:
dates.dt.day_name('es_ES')

0           martes
1        miércoles
2           jueves
3          viernes
4           sábado
           ...    
14155       martes
14156    miércoles
14157       jueves
14158      viernes
14159       sábado
Name: DATE, Length: 14160, dtype: string[pyarrow]

In [55]:
dates.dt.is_month_end

0        False
1        False
2        False
3        False
4        False
         ...  
14155    False
14156    False
14157    False
14158    False
14159    False
Name: DATE, Length: 14160, dtype: bool[pyarrow]

In [56]:
dates.dt.strftime('%d/%m/%y')

0        01/01/80
1        02/01/80
2        03/01/80
3        04/01/80
4        05/01/80
           ...   
14155    03/09/19
14156    04/09/19
14157    05/09/19
14158    06/09/19
14159    07/09/19
Name: DATE, Length: 14160, dtype: string[pyarrow]

In [59]:
classes = ['cs106', 'cs150', 'hist205', 'hist206', 'hist207']
start_dates = (pd.Series(['2015-03-08','2015-03-08','2015-03-09','2015-03-09','2015-03-11'],
                          dtype='datetime64[ns]', index=classes)
                          .astype('timestamp[ns][pyarrow]'))
start_dates

cs106      2015-03-08 00:00:00
cs150      2015-03-08 00:00:00
hist205    2015-03-09 00:00:00
hist206    2015-03-09 00:00:00
hist207    2015-03-11 00:00:00
dtype: timestamp[ns][pyarrow]

In [69]:
classes = ['cs106', 'cs150', 'hist205', 'hist206', 'hist207']
end_dates = (pd.Series(['2015-05-28 23:59:59',
                        '2015-06-01 3:00:00',
                        '2015-06-03',
                        '2015-06-02 14:20',
                        '2015-06-01'],
                       dtype='datetime64[ns]', index=classes)
                       .astype('timestamp[ns][pyarrow]'))

end_dates


cs106      2015-05-28 23:59:59
cs150      2015-06-01 03:00:00
hist205    2015-06-03 00:00:00
hist206    2015-06-02 14:20:00
hist207    2015-06-01 00:00:00
dtype: timestamp[ns][pyarrow]

In [71]:
duration = (end_dates - start_dates)
duration

cs106      81 days 23:59:59
cs150      85 days 03:00:00
hist205    86 days 00:00:00
hist206    85 days 14:20:00
hist207    82 days 00:00:00
dtype: duration[ns][pyarrow]

In [73]:
duration.astype('timedelta64[ns]')
duration

cs106      81 days 23:59:59
cs150      85 days 03:00:00
hist205    86 days 00:00:00
hist206    85 days 14:20:00
hist207    82 days 00:00:00
dtype: duration[ns][pyarrow]

In [74]:
(duration
 .astype('timedelta64[ns]')
 .dt.total_seconds()
 )

cs106      7084799.0
cs150      7354800.0
hist205    7430400.0
hist206    7395600.0
hist207    7084800.0
dtype: float64

In [75]:
(duration
 .astype('timedelta64[ns]')
 .dt.seconds)

cs106      86399
cs150      10800
hist205        0
hist206    51600
hist207        0
dtype: int32

In [79]:
(duration
 .astype('timedelta64[ns]')
 .dt.days)

cs106      81
cs150      85
hist205    86
hist206    85
hist207    82
dtype: int64

In [80]:
# Exercises

In [96]:
# Convert a column with date information to a date
url = 'https://github.com/mattharrison/datasets/raw/master/data/alta-noaa-1980-2019.csv'
alta_df = pd.read_csv(url)

dates = (pd.to_datetime(alta_df.DATE)
 .astype('datetime64[ns]'))

In [107]:
# Convert a date column into UTC dates

dates_conv = (dates.dt.tz_convert("America/Denver"))
dates_conv

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

In [123]:
# Convert a date column into local dates with a timezone

dates_local = (dates_conv.dt.tz_convert('UTC'))
dates_local

0       1980-01-01 07:00:00+00:00
1       1980-01-02 07:00:00+00:00
2       1980-01-03 07:00:00+00:00
3       1980-01-04 07:00:00+00:00
4       1980-01-05 07:00:00+00:00
                   ...           
14155   2019-09-03 06:00:00+00:00
14156   2019-09-04 06:00:00+00:00
14157   2019-09-05 06:00:00+00:00
14158   2019-09-06 06:00:00+00:00
14159   2019-09-07 06:00:00+00:00
Name: DATE, Length: 14160, dtype: datetime64[ns, UTC]

In [127]:
# Convert a date column into epoch values

dates_epoch = dates_conv.astype('int64[pyarrow]')
dates_epoch

0         315558000000000000
1         315644400000000000
2         315730800000000000
3         315817200000000000
4         315903600000000000
                ...         
14155    1567490400000000000
14156    1567576800000000000
14157    1567663200000000000
14158    1567749600000000000
14159    1567836000000000000
Name: DATE, Length: 14160, dtype: int64[pyarrow]

In [130]:
# Convert an epoch number into UTC

dates_UTC = dates_epoch.astype('datetime64[ns]')
dates_UTC

0       1980-01-01 07:00:00
1       1980-01-02 07:00:00
2       1980-01-03 07:00:00
3       1980-01-04 07:00:00
4       1980-01-05 07:00:00
                ...        
14155   2019-09-03 06:00:00
14156   2019-09-04 06:00:00
14157   2019-09-05 06:00:00
14158   2019-09-06 06:00:00
14159   2019-09-07 06:00:00
Name: DATE, Length: 14160, dtype: datetime64[ns]